In [ ]:
import os 
import json
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from utils import *


In [ ]:
from img_utils import *

# Loading Data 

In [ ]:
path = "../nybolig-scrape/output"
houses_df = data_to_DF(path, max_houses=1000)
houses_df = houses_df[(houses_df['postal_code'] >= 1000) & (houses_df['postal_code'] <= 2920)]
houses_df = houses_df[houses_df['type'] == 'ejerlejlighed']
print("Number of datapoints of type 'Ejerlejlighed': ", len(houses_df))
display(houses_df)

# Setting up splits

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train, validation and test sets with a 60-20-20 ratio
train_df, test_df = train_test_split(houses_df, test_size=0.2, random_state=0)
train_df, valid_df = train_test_split(train_df, test_size=0.20, random_state=0)

In [ ]:
# As most of the original images are 2100x1400, we resize them to 448*x
target_width = 224*3
target_height = 224*3

resize = True
gray_scale = False
threshhold = True

train_images_RGB = preprocess_images(train_df, "image_floorplan", target_width, target_height, resize, gray_scale, threshhold)
validation_images_RGB = preprocess_images(valid_df, "image_floorplan", target_width, target_height, resize, gray_scale, threshhold)
test_images_RGB = preprocess_images(test_df, "image_floorplan", target_width, target_height, resize, gray_scale, threshhold)

train_prices = train_df['price']
validation_prices = valid_df['price']
test_prices = test_df['price']

#Plot the first 5 images and their prices 
fig, ax = plt.subplots(1, 5, figsize=(20, 20))
for i in range(5):
    ax[i].imshow(train_images_RGB[i])
    ax[i].set_title(f"Price: {train_prices.iloc[i]}")
    ax[i].axis('off')
plt.show()


# Pre-trained VGG16

In [ ]:
import tensorflow as tf
# Check available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Set the GPU to be used
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU available")

## Model Fitting

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Load pre-trained VGG16 model (without including top layers)
input_shape = train_images_RGB[0].shape 
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add new top layers for regression
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation="relu"),
    Dense(256, activation="relu"),
    Dense(1, activation="linear")
])

In [ ]:
# Compile the model
#0.0001
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_absolute_error')
model.summary()

In [ ]:
# Train the model
history = model.fit(train_images_RGB, train_prices, validation_data=(validation_images_RGB, validation_prices), epochs=2)
#save_model(model, "VVG16_model")

In [ ]:
model.evaluate(test_images_RGB, test_df["price"])

# Make predictions
predictions = model.predict(test_images_RGB)

## Model Evaluation

In [ ]:
real_prices = test_df['price'].values
predicted_prices = predictions.flatten()

# Print the R2 score, MAE and MSE
print(f"R2 score: {r2_score(real_prices, predicted_prices):.2f}")
print(f"Mean Absolute Error: {mae(real_prices, predicted_prices):.2f}")
print(f"Mean Squared Error: {mse(real_prices, predicted_prices):.2f}")

# Plot the predictions
plt.figure(figsize=(10, 10))
for i, (image, label, prediction) in enumerate(zip(test_images_RGB[0:9], test_df["price"][0:9], predictions[0:9])):
    plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(f"Real: {label}\nPredicted: {prediction[0]:.0f}")
    plt.axis("off")
plt.show()

# Plot the predictions vs real prices
plot_regression_results('VGG16', real_prices, predicted_prices)